---
# Stochastic Gradient Descent for Logistic Regression
### An assignment as part of the course INF367: Selected Topics in Artificial Intelligence at the University of Bergen

### Description of the problem
We want to discover the relationship between the features of some observation(s) and some outcome, e.g. a classification. Given enough samples of sufficient quality, we can train a probabilistic machine learning classifier to predict the outcome for new samples. Such a classifier generally has four components: 

1. A feature representation of the input. 
    For each input observation $x^i$, this will be a vector of features $[x_1 , x_2 , ..., x_n]$.
2. A classification function that computes $ŷ$, the estimated class, via $p(y|x)$. 
3. An objective function for learning, usually involving minimizing error on
training examples.
4. An algorithm for optimizing the objective function. <br>
(Jurafsky & Martin, 2022)

This report will take a closer look at an algorithm for component 4, namely stochastic gradient descent for binary logistic regression. 


### Description of the approach
This report provides and describes an implementation of the stochastic gradient descent algorithm detailed on page 93 of [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/ed3book.pdf), Third Edition, by Daniel Jurafsky and James H. Martin, which can be seen in the figure below:

<div>
<img src="sgd_alg_jurafsky.png" width="600"/>
</div>

<div align="center"><i><b>Fig. 3</b>: The Stochastic Gradient Descent Algorithm. (Jurafsky & Martin, 2022)<br></i></div>

_Step 1 (computing the loss) is used mainly to report how well we are doing on the current tuple; we don’t need to compute the loss in order to compute the gradient. The algorithm can terminate when it converges (or when the gradient norm $<\epsilon$), or when progress halts (for example when the loss starts going up on a held-out set)._ (Jurafsky & Martin, 2022)

My approach differs somewhat from the description above. The main difference is that my implementation of stochastic gradient descent doesn't take the loss function $L(f(x; \theta), y)$
    and the function parametrized by $\theta$ $f(x, \theta)$ as input. 
    Since this implementation is for binomial logistic regression, I've added $L$ and $f$ into the class methods.
    The cross-entropy loss function $L(\sigma(w \cdot x + b) - y) * x^i)$ is part of the `gradient` method,
    while the function parametrized by $\theta$ is the sigmoid function implemented in the `sigmoid` method.

### Description of the software
There are two ways to run the software: <br>
**1. As part of this Jupyter Notebook.** <br>
Open this file in Jupyter Notebook or an IDE capable of running notebooks (e.g. PyCharm). Select "Kernel" from the top menu, then "Restart and Run All" to run the notebook in your local environment. 
Since there are no dependencies other than `numpy` required to run the code, re-use through copy-pasting the code is also trivial. Simply copy-paste the code over to your Python file and you should be able to run it. 
**2. By pip installation** <br>
A directory called `nb_sgd_classify` has been provided along with this notebook. To install the software, simply download the folder, navigate to it in the terminal, and run the command `python3 -m pip install .` (punctuation included. Indicates that pip should install the current directory.) This will install both the software package and required dependencies (numpy), unless these are already installed. Then you may use the SGD and Naive Bayes classes as you wish by importing them as usual. See the `test` directory for example usage. 

<br>
My version of the stochastic gradient descent algorithm is implemented as a Python class with three methods: <br>
1. `sigmoid`: The logistic function. <br>
2. `gradient`: Calculates the loss, and the gradient of the loss function. <br>
3. `train`: Runs the gradient descent algorithm on the training data. <br>

The class does not have a test method, because this wasn't required in the assignment text, and I ran out of time. I do, however, aim to implement a test method in the future as part of a complete implementation of a logistic regression classifier.

In [7]:
import math
import random
import numpy as np
from typing import Union

In [8]:
class SGD:
    """
    An implementation of the stochastic gradient descent algorithm for binomial logistic regression.
    
    I used the algorithm description in 'Speech and Language Processing' (Jurafsky & Martin, 2022), page 93,
    as a guideline for this implementation. 
    
    The main difference between this algorithm and the one outlined
    by Jurafsky & Martin is that this algorithm doesn't take the loss function 'L(f(x; theta), y)'
    and the function parametrized by theta 'f(x, theta)' as input. Since this implementation
    is for binomial logistic regression, I've added L and f into the class methods.
    The cross-entropy loss function L (sigmoid(w.dot(x) + b) - y) * x[i]) is part of the gradient() method,
    while the function parametrized by theta is the sigmoid function implemented in the sigmoid() method.
    """
    
    def __init__(self, random_state: int = 42):
        self.theta = None
        self.b = 0
        self.w = None
        self.random_state = random_state
        self.epoch = 0

    @classmethod
    def sigmoid(cls, z: float):
        """
        The logistic function.
        Maps a real value to the range [0, 1].
        Used to calculate the gradient during gradient descent.
        :param z: A floating point (real) value
        :return: A floating point value in the range [0, 1].
        """
        return 1 / (1 + math.exp(-z))

    @classmethod
    def gradient(cls, x: np.ndarray, y: Union[int, float], theta: np.ndarray):
        """
        Calculates the loss, and the gradient of the loss function.
        :param x: A numpy array. Represents a single training sample.
        :param y: A scalar. Represents the label for the training sample (0 or 1).
        :param theta: A numpy array. Represents the current values for the weights.
        :return: A numpy array of size len(x) + 1. Represents the gradient of the training sample and the bias.
        """
        # Extracts the weights from the parameter vector
        w = np.delete(theta, -1)
        # Extracts the bias from the parameter vector
        b = theta[-1]
        # Calculates the loss for each weight and then calculates the derivative
        delta_w = np.array([(cls.sigmoid(w.dot(x) + b) - y) * x[i] for i in range(len(x))])
        # Calculates the loss for the bias and then calculates the derivative
        delta_b = cls.sigmoid(w.dot(x) + b) - y
        # Extends the derivative of the bias with the derivative of the weights
        delta = np.append(delta_w, delta_b)
        # Returns the derivative (gradient) of the (entire) loss function
        return delta

    def train(self, X_train: np.ndarray, y_train: np.ndarray, learning_rate: float = 0.01, max_iter: int = 1000, tolerance: float = 0.0001):
        """
        Runs the gradient descent algorithm on the training data.
        :param X_train: A numpy array of arrays, where each subarray contains the feature values
                        for a single training sample.
        :param y_train: A numpy array containing the values for the label of each training sample.
        :param learning_rate: The value of the learning rate hyperparameter,
                              a.k.a. the "step size" for gradient descent.
        :param max_iter: The maximum number of epochs (passes/iterations) to performed during gradient descent.
        :param tolerance: The minimum
        :return: Theta, i.e. the optimal parameters for the weights w and the bias b.
        """

        # Basic error handling for user input
        if len(X_train) == 0:
            raise ValueError("X_train must have at least 1 sample. ")
        if len(X_train) != len(y_train):
            raise ValueError("X_train and y_train must be of the same length.")
        if learning_rate <= 0 or max_iter <= 0 or tolerance <= 0:
            raise ValueError("The hyperparameters 'learning_rate', 'max_iter' and 'tolerance' must be greater than 0.")

        # Initializes the weights and the bias to 0
        self.theta = np.array([0 for i in range(len(X_train[0]))] + [self.b])
        print(f"Number of parameters to tune: {len(self.theta)}")

        # Randomly selects a set of indices of training samples to use for SGD algorithm
        random.seed(self.random_state)
        random_indices = random.sample([i for i in range(len(X_train))], len(X_train))

        # Performs gradient descent with max_iter number of steps towards the optimum
        for epoch in range(max_iter + 1):
            self.epoch += 1
            # Randomly selects a sample
            for i in random_indices:
                # Calculates the gradient
                delta = self.gradient(X_train[i], y_train[i], self.theta)
                # Checks whether the change in weights will be greater than the tolerance
                if np.all(np.abs(learning_rate * delta) <= tolerance):
                    # If it isn't, stop the gradient descent and return the parameters tuned after the previous epoch
                    print("\n", f"""
                    The optimal parameters after {self.epoch} epochs are:
                    w1 = {self.theta[0]}
                    w2 = {self.theta[1]}
                    b = {self.theta[2]}
                    """)
                    return self.theta
                # Updates the weights
                else:
                    self.theta = self.theta - (learning_rate * delta)
            if epoch == 0 or (epoch + 1)% 10 == 0:
                print(f"Parameters after epoch {epoch + 1}: {self.theta}")

        print("\n", f"""
        The optimal parameters after {self.epoch} epochs are:
        w1 = {self.theta[0]}
        w2 = {self.theta[1]}
        b = {self.theta[2]}
        """)
        
        return self.theta

### Testing the implementation on a single training example

In [9]:
# Instantiating and running Stochastic Gradient Descent on a single training example
X_train = np.array([[3, 2]])  # A single training example
y_train = np.array([1])  # The label for our training example (1 = "positive sentiment", 0 = "negative sentiment")
sgd = SGD(random_state=42)
theta = sgd.train(X_train, y_train, learning_rate=0.01, max_iter=500, tolerance=0.001)

Number of parameters to tune: 3
Parameters after epoch 1: [0.015 0.01  0.005]
Parameters after epoch 10: [0.12873805 0.08582537 0.04291268]
Parameters after epoch 20: [0.22152599 0.14768399 0.073842  ]
Parameters after epoch 30: [0.2917358  0.19449053 0.09724527]
Parameters after epoch 40: [0.3473211 0.2315474 0.1157737]
Parameters after epoch 50: [0.39294022 0.26196014 0.13098007]
Parameters after epoch 60: [0.43143619 0.28762413 0.14381206]
Parameters after epoch 70: [0.46463256 0.30975504 0.15487752]
Parameters after epoch 80: [0.49375382 0.32916922 0.16458461]
Parameters after epoch 90: [0.51965545 0.34643697 0.17321848]
Parameters after epoch 100: [0.54295553 0.36197035 0.18098518]
Parameters after epoch 110: [0.56411354 0.37607569 0.18803785]
Parameters after epoch 120: [0.58347949 0.38898632 0.19449316]
Parameters after epoch 130: [0.60132559 0.40088373 0.20044186]
Parameters after epoch 140: [0.61786746 0.41191164 0.20595582]
Parameters after epoch 150: [0.63327855 0.4221857  0

### Testing the implementation on a multiple training examples

In [10]:
# Instantiating and running Stochastic Gradient Descent on a multiple training examples
X_train = np.array([[3, 2], [4, 1], [5, 0], [2, 1], [1, 0], [0, 1], [3, 6], [2 ,8], [3, 3], [1, 8]])
y_train = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])  # The label for our training examples
sgd = SGD(random_state=42)
theta = sgd.train(X_train, y_train, learning_rate=0.01, max_iter=500, tolerance=0.001)

Number of parameters to tune: 3
Parameters after epoch 1: [ 0.03255003 -0.10212835  0.00107459]
Parameters after epoch 10: [ 0.33036689 -0.43561577  0.06030454]
Parameters after epoch 20: [ 0.51664447 -0.61571864  0.10372972]

 
                    The optimal parameters after 23 epochs are:
                    w1 = 0.5661911317529388
                    w2 = -0.6359628022501363
                    b = 0.11921399347902256
                    
